<a href="https://colab.research.google.com/github/mohammad-rahbari/federated-learning_visual-classification/blob/mmd_branch/notebooks/federated_model_visual_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing DINO and installing its dependencies

In [6]:
# @title Clon the DINO ripo
!git clone https://github.com/facebookresearch/dino.git

Cloning into 'dino'...
remote: Enumerating objects: 175, done.
remote: Total 175 (delta 0), reused 0 (delta 0), pack-reused 175 (from 1)
Receiving objects: 100% (175/175), 24.47 MiB | 42.32 MiB/s, done.
Resolving deltas: 100% (100/100), done.


In [7]:
# @title Installing required dependencies regarding DINO
%cd dino
!pip install -r requirements.txt
!pip install timm

/content/dino/dino
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'



# preprocessing the CIFAR-100 dataset

feature size in CIFAR is 32x32 but DINO requires 224x224 in the input layer.

In first step we upscale the dataset and then we add randomization to it

In last step of transformation we normalize data usind mean value and standard division of ImageNet



In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split,DataLoader

In [9]:
# transform_train = transforms.Compose([
#     transforms.Resize(256),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                          std=(0.229, 0.224, 0.225))
# ])

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                         std=(0.229, 0.224, 0.225))
])





In [34]:
from torch.utils.data import ConcatDataset
from torchvision.datasets import CIFAR100
# train_dataset = torchvision.datasets.CIFAR100(
#     root='./data', train=True, download=True, transform=transform)

# test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
#                                        download=True, transform=transform)


# full_train = ConcatDataset([train_dataset, test_dataset])


class ExtendedCIFAR100(CIFAR100):
    def __init__(self, root, train=True, transform=None, download=False):
        super().__init__(root=root, train=train, transform=transform, download=download)

        # Load the test dataset
        test_dataset = CIFAR100(root=root, train=False, download=download)

        # Concatenate data using numpy
        self.data = np.concatenate((self.data, test_dataset.data), axis=0)

        # Concatenate targets using list concatenation
        self.targets = self.targets + test_dataset.targets

        self.indices = list(range(len(self.data)))

full_train = ExtendedCIFAR100(root='./data', train=True, transform=transform, download=True)


# Verify the length of the new dataset
print(f"Length of combined dataset: {len(full_train)}")


Length of combined dataset: 60000


In [11]:
#@title Imports
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import numpy as np
import random
import torch

# Set Hyperparameters regarding the data spliting here!


# DO NOT FOREGET TO TEST K-FOLD SLPITTING !!!!!

In [28]:

#@title set the parameters here!!



number_of_clients = None
train_frac = 0.8 #@param
val_frac = 0.2 #@param
is_seed_fixed = True #@param{type:"boolean"}
seed = 42 #@param{type:"integer"}

def set_seed(seed=42, is_seed_fixed=True):
  if not is_seed_fixed:
    return
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  random.seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False


set_seed(seed,is_seed_fixed)



#@markdown </br> <h5>Indicate the number of clients that contribute in training:</h5>
n_clients = 10 #@param{type:"integer"}

#@markdown </br></br> <b>splitting hyperparameters</b>

spliting_method = "non-i.i.d. sharing" #@param["i.i.d. sharing","non-i.i.d. sharing"]




In [29]:
#@title Set the parameters here only if <b>non-i.i.d. sharing</b> method had been selected!!
#@markdown Nc is the number of classes that each subset can contain
if spliting_method == "non-i.i.d. sharing":
  Nc = 70 #@param{type:"integer"}

  # are_classes_overlaping = False #@param{type:"boolean"}

#@markdown <h3>If we consider the Number of classes M and nummber of client K then:</h3>
#@markdown <ul>
#@markdown   <li>Nc should be:
#@markdown     <ul>
#@markdown       <li>
#@markdown         Greater than or equal to <b>\\(\frac{M}{K}\\)</b>
#@markdown       </li>
#@markdown       <li>
#@markdown         Less than or equal to K </b>
#@markdown       </li>
#@markdown     </ul>
#@markdown   </li>
#@markdown   <li>
#@markdown   Muximum number of clients means all classes contribute in every client
#@markdown   </li>

#@markdown </ul>


#@markdown </br></br><h3>Combination of classes are randomly selected which suits definition of federated learning especially Cross-device federated learning</h3>





# Data splitting

In [35]:
# @title data splitting

set_seed(seed,is_seed_fixed)
generator = torch.Generator().manual_seed(seed)

total_size = len(full_train)
train_size = int(train_frac * total_size)
val_size   = total_size - train_size

train_set, val_set = random_split(full_train, [train_size, val_size], generator=generator)
train_indices = torch.tensor(train_set.indices)
val_indices = torch.tensor(val_set.indices)

train_set = Subset(train_set.dataset, train_indices)
val_set = Subset(val_set.dataset, val_set.indices)

train_loader = DataLoader(train_set, batch_size=len(train_set), shuffle=False)
val_loader  =  DataLoader(val_set, batch_size=len(val_set), shuffle=False)



print(f"Train dataset size: {len(train_set)}")
print(f"Validation dataset size: {len(val_set)}")

lenghts = [train_size//n_clients] * n_clients

for i in range(train_size % n_clients):
  lenghts[i] += 1
print("Size of subset: ", lenghts)





Train dataset size: 48000
Validation dataset size: 12000
Size of subset:  [4800, 4800, 4800, 4800, 4800, 4800, 4800, 4800, 4800, 4800]


In [43]:
# @title i.i.d sharing - split data dased on number of clients and with respect of label proportionality
set_seed(seed,is_seed_fixed)
def iid_sharing(dataset, n_clients):

  full_train_indices = dataset.indices
  full_train_labels = torch.from_numpy(np.array(dataset .dataset.targets)[full_train_indices]) #collects labels from all dataset
  unique_lables = torch.unique(full_train_labels) #Removes dupilication and generates a uniuqe list of labels (classes)
  proportionality ={}
  classes_indices = {}


  for i in unique_lables:
    proportionality[i] =( full_train_labels == i).sum() / len(full_train_labels) #Calculates proportinality of each class
    classes_indices[i] = torch.nonzero(full_train_labels == i).squeeze() #Collects and save Indices in an array based on classes

  for i in classes_indices.keys():
    classes_indices[i] = classes_indices[i][torch.randperm(classes_indices[i].shape[0])] #suffels the indices

  client_data_size = len(full_train_labels) / n_clients #Minimum dataset size of each client

  client_indices = {}

  #For each client we generate a element in client_indices dict to keep track of indices we'll associated with each client
  for client in range(n_clients):
    if not client_indices.get(client):
      client_indices[client] = torch.empty(0, dtype=torch.long)
  #__________________


  #For each client we calculate how many samples from each specific label should be seperated. We take out the requried number of them form the list
    for label in proportionality.keys():
      pointer = proportionality[label] * client_data_size
      pointer = int(pointer) if not pointer % 1 else int(pointer) + 1
      pointer = min(pointer,classes_indices[label].size()[0])
      pointer = pointer if pointer < classes_indices[label].size()[0] else classes_indices[label].size()[0]
      client_indices[client] = torch.cat((client_indices[client], classes_indices[label][:pointer]), dim=0)
      classes_indices[label] = classes_indices[label][pointer:]


  #After spliting data we distribute remaining samples amoung the clients
  for label in classes_indices.keys():
    while True:
      for client in client_indices.keys():

        if classes_indices[label].size()[0] == 0:
          break
        client_indices[client] = torch.cat((
            client_indices[client],
            classes_indices[label][:1] ),
            dim=0)


        classes_indices[label] = classes_indices[label][1:]

      if classes_indices[label].size()[0] == 0:
        break


  #spilt actual dataset to mulitiple subset for clients
  client_data={
      client_id: Subset(dataset.dataset,indices[torch.randperm(len(indices))])
      for client_id, indices in client_indices.items()
  }
  return client_data


indices_check = []
client_data = iid_sharing(train_set, n_clients)
for client_id in client_data.keys():
  indices_check = indices_check + list(client_data[client_id].indices)
  print(f"Client {client_id} has {len(client_data[client_id])} samples")



Client 0 has 4847 samples
Client 1 has 4847 samples
Client 2 has 4847 samples
Client 3 has 4847 samples
Client 4 has 4847 samples
Client 5 has 4847 samples
Client 6 has 4847 samples
Client 7 has 4847 samples
Client 8 has 4847 samples
Client 9 has 4377 samples


In [44]:
# @title Non i.i.d sharing


# @title i.i.d sharing - split data dased on number of clients and with respect of label proportionality
set_seed(seed,is_seed_fixed)
def noniid_sharing(dataset,Nc , n_clients):

  full_train_indices = dataset.indices
  full_train_labels = torch.from_numpy(np.array(dataset .dataset.targets)[full_train_indices]) #collects labels from all dataset
  unique_lables = torch.unique(full_train_labels) #Removes dupilication and generates a uniuqe list of labels (classes)

  classes_indices = {}
  classes_size = torch.zeros(unique_lables.size()[0])

  class_combs = get_class_combinations(unique_lables, Nc, n_clients)

  classes_num_partition = torch.zeros(unique_lables.size()[0])

  for i in unique_lables:
    classes_num_partition[i] = torch.sum(class_combs == i)
    classes_indices[i.item()] = torch.nonzero(full_train_labels == i).squeeze() #Collects and save Indices in an array based on classe
    classes_size[i] = classes_indices[i.item()].size()[0] #Calculate the number of smaples belonging to each class

  for i in classes_indices.keys():
    classes_indices[i] = classes_indices[i][torch.randperm(classes_indices[i].shape[0])] #suffels the indices

  client_indices = {}

  #For each client we generate a element in client_indices dict to keep track of indices we'll associated with each client

  for client in range(n_clients):
    client_indices[client] = torch.tensor([],dtype=torch.int16)
    for cls in class_combs[client]:
      cls = cls.item()

      class_partition = torch.empty(0, dtype=torch.int64)

      portion  = classes_size[cls] /classes_num_partition[cls]
      portion = int(portion) if not portion % 1 else int(portion) + 1

      if portion < classes_indices[cls].size()[0]:
        class_partition = classes_indices[cls][:portion]
        classes_indices[cls] = classes_indices[cls][portion:]
      else:
        class_partition = classes_indices[cls]
        classes_indices[cls] = torch.tensor([])
      client_indices[client] = torch.cat((client_indices[client], class_partition), dim=0)

  client_data={
      client_id: Subset(dataset.dataset,indices[torch.randperm(len(indices))])
      for client_id, indices in client_indices.items()
  }


  check_list = {}

  return client_data, class_combs





def get_class_combinations(classes, Nc, n_clients):

  if Nc * n_clients < len(classes):
    Nc = len(classes) / n_clients
    Nc = int(Nc) if not Nc % 1 else int(Nc) + 1

    print(f"Number of classes per clients is lower then minimum. Nc changed to {Nc} (the least possible value)")

  combinations = torch.zeros((n_clients,Nc),dtype= torch.int64)
  counter =0
  ofset = 0
  flag = False

  for i in range(n_clients):
    if not flag:
      end_pointer = (i + 1) * Nc
      if end_pointer >= classes.size()[0]:
          ofset = (end_pointer - classes.size()[0])
          flag = True

      combinations[i] = classes[i* Nc - ofset: end_pointer - ofset]

    else:

      combinations[i]  = torch.randperm(classes.size()[0])[:Nc]






  return combinations




client_data, class_combs = noniid_sharing(train_set,5, 25)


# Log System

In this section Requerd Data will be archaved.<br/><br/>
**Archaving this inforamtion will make it possible to:**
*   Handel Clients
*   Manage the models
*   Keep track of results of different Backbones
*   Compare measurement criteria
*   Handel model merging process
*   Save path to the models

<br/><br/>
**These data will be saved in two seperted csv file to :**

1.   Archave the LOCAL Models  
2.   Archave the GLOBAL Models resulted by each round

<br/><br/>
The csv files will be handeled as panda.dataframe and each row in the csv file addresses one of models
<br/>

**Columns (COMMON):**<br/>
1. Backbone model name
2. Model name
3. Path
4. Time of log
5. Measurement criteria
 * loss
 * Accuracy
 * ...?
6. Size of dataset

**Columns (Local Models only):**<br/>
7. Client Id
8. Classes (Indicate which classes have been covered by each client)(format:"2,4,63,80,9" or "all" for all the classes)
9. Round number
10. Duration of training
11. Train Test ratio

**Columns (Global Models only):**<br/>
7. Number of clients
7. Number of rounds
8. Model Aggregation method







In [ ]:
# @title Functions
import torch
from datetime import datetime
import time
from google.colab import drive


def get_current_time():
  now = datetime.now()

  formatted_date_time = now.strftime("%Y-%m-%d %H:%M:%S") # Format the date and time as a string

  return formatted_date_time



tic_start_time = None

def tic():
    global tic_start_time
    tic_start_time = time.perf_counter() # start the timer

def toc():
    if tic_start_time is None:
        print("Error: You must call tic() before toc()")
        return None
    elapsed_time = time.perf_counter() - tic_start_time
    return elapsed_time


def load_storage():
  drive.mount('/content/drive')
  path_to_clients = '/content/drive/MyDrive/MLDL_FederatedLearning/models/clients/'
  path_to_global  = '/content/drive/MyDrive/MLDL_FederatedLearning/models/global/'
  return path_to_clients, path_to_global




def save_client(model,client,round_number,backbone,path_to_clients):
  round_number = str(round_number)
  round_number = "0"*(4-len(round_number)) + round_number

  client = str(client)
  client = "0"*(4-len(client)) + client

  model_name = backbone + "_" + client + "_" + round_number + ".pth"

  path =  path_to_clients + model_name

  numbertorch.save(model.state_dict(),  path )

  return model_name, path




def save_global_model(model,number_of_clients,number_of_round,backbone,path_to_global):
  number_of_round = str(number_of_round)
  number_of_round = "0"*(4-len(number_of_round)) + number_of_round

  number_of_clients = str(number_of_clients)
  number_of_clients = "0"*(4-len(number_of_clients)) + number_of_clients

  model_name = backbone + "_" + number_of_clients + "_" + number_of_round + ".pth"

  path =  path_to_global + model_name

  numbertorch.save(model.state_dict(), path  )

  return model_name, path
